In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from huggingface_hub import login, HfApi
import os

#-----------------------------------------
# 2. 登录 Hugging Face
login(token="~")

base_model_path = "Qwen/Qwen2.5-7B-Instruct"
epoch = 2200

lora_ckpt_path = f"outputs/Qwen2.5-7B-Instruct/checkpoint-{epoch}"
base_model = AutoModelForCausalLM.from_pretrained(base_model_path)
model = PeftModel.from_pretrained(base_model, lora_ckpt_path)
merged_model = model.merge_and_unload()
merged_model.half()
save_path = f"merged_qwen_7B_model_-{epoch}"
merged_model.save_pretrained(save_path)
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
tokenizer.save_pretrained(save_path)
repo_id = f"hf_id/Qwen2.5-7B-Instruct-{epoch}"

api = HfApi()
api.create_repo(repo_id=repo_id, exist_ok=True)
api.upload_folder(
    folder_path=save_path,
    repo_id=repo_id,
    commit_message=f"Upload merged Qwen-7B model at epoch {epoch}"
)
print("模型上传成功 →", repo_id)